In [2]:
import pandas as pd
from pandas.plotting import _converter
import re

In [2]:
pat = re.compile('RT @?([A-z0-9_]+):')

In [3]:
url = re.compile('https?:.*')

In [4]:
emoji_pattern = re.compile(r'\\[u|U][a-fA-F0-9]+[\\u|U[a-zA-F0-9]+]*')

In [5]:
df  = pd.read_csv('Replies_WorldCupDraw.csv',encoding='utf8')
df.head()

,in_reply_to,text,emojis_used,retweet_count,like_count
0,937247505208827904,MOTD couldn’t find anything negative to say ab...,NaN,0,1
1,937247505208827904,7 months of this nonsense to come,😴,0,1
2,937247505208827904,If he did that penalty against me first chance...,NaN,0,1
3,937247505208827904,He another world class,NaN,0,0
4,937247505208827904,I bet they are,NaN,0,0


In [6]:
df.shape

(1081697, 5)

In [7]:
df.rename(columns=
         {
             'in_reply_to':'id'
         },inplace = True)

In [8]:
#removing empty tweets
df = df[df['text'].str.strip() != '']
df.shape

(1081696, 5)

In [9]:
# Total number of retweets in replies
df[df.text.str.contains('RT @[A-z0-9]+:')].shape

(25, 5)

In [10]:
# Test for emoji
# sample = 'ربنا يوفق صلاح وباقي افراد المنتخب المصري'
# t = ascii(sample)
# emojis_used = emoji_pattern.findall(t)
# emojis = list()
# for u in emojis_used:
#      a = u.encode('utf8')
#     emojis.append(a.decode('unicode-escape'))
# emojis

In [11]:
RT = []
Text = []
OTweeter = []

for text in df.text:
    retweet = text.split()[0]=='RT'
##print(t,retweet)
    RT.append(retweet)
    if retweet:
        OTweeter.append(pat.match(text).group(1) if pat.match(text) else '')
        text = re.sub(pat,"",text)
    else:
        OTweeter.append('')
    text = ascii(text)
    emojis_used = emoji_pattern.findall(text)
    emojis = list()
    for u in emojis_used:
        a = u.encode('utf8')
        emojis.append(a.decode('unicode-escape'))
    
    emojis_used = "".join(emojis)
    text = re.sub(emoji_pattern,"",text)
    text = re.sub(url,"",text)    
    text = text.strip('\'')
    text = text.strip('\"\"\"')
    Text.append(text.strip())

In [12]:
df['RT'] = RT
df['text'] = Text
df['original_tweeter'] = OTweeter
df.head()

,id,text,emojis_used,retweet_count,like_count,RT,original_tweeter
0,937247505208827904,MOTD couldn find anything negative to say abou...,NaN,0,1,False,
1,937247505208827904,7 months of this nonsense to come,😴,0,1,False,
2,937247505208827904,If he did that penalty against me first chance...,NaN,0,1,False,
3,937247505208827904,He another world class,NaN,0,0,False,
4,937247505208827904,I bet they are,NaN,0,0,False,


In [13]:
df = df[df['text'].str.strip() != '']
df.shape

(1061657, 7)

In [14]:
df.dtypes

id                   int64
text                object
emojis_used         object
retweet_count        int64
like_count           int64
RT                    bool
original_tweeter    object
dtype: object

In [15]:
df.to_csv('Replies_WorldCupDraw_Cleansed.csv',encoding='utf8',index=False)

In [16]:
org_tweet = pd.read_csv('WorldCupDraw_Cleansed.csv',encoding='utf8')
org_tweet.shape

(106548, 11)

In [17]:
dataset = pd.merge(org_tweet,df,how='inner',on='id',sort=False)
dataset.shape

(1058559, 17)

In [18]:
dataset.head(n=2)

,id,text_x,screen_name,retweet_count_x,favorite_count,friends_count,followers_count,url,RT_x,original_tweeter_x,emojis_used_x,text_y,emojis_used_y,retweet_count_y,like_count,RT_y,original_tweeter_y
0,937247505208827904,Eden Hazard says the England and Belgium playe...,SPonTheBall,35,0,81,25,https://t.co/0Ekz6haggd,True,BBCMOTD,NaN,MOTD couldn find anything negative to say abou...,NaN,0,1,False,
1,937247505208827904,Eden Hazard says the England and Belgium playe...,SPonTheBall,35,0,81,25,https://t.co/0Ekz6haggd,True,BBCMOTD,NaN,7 months of this nonsense to come,😴,0,1,False,


In [19]:
dataset[dataset.id == 937247505208827904].shape

(6, 17)

In [20]:
dataset[dataset.id == 937247505208827904].text_y

0    MOTD couldn find anything negative to say abou...
1                    7 months of this nonsense to come
2    If he did that penalty against me first chance...
3                               He another world class
4                                       I bet they are
5                             Hazard has always right.
Name: text_y, dtype: object

In [21]:
stat_dataset = dataset.groupby('id')

In [22]:
# %matplotlib inline
# stat_dataset.count().plot(kind='bar')

In [23]:
number_of_replies = stat_dataset.id.count()
avg_reply_fav_count = stat_dataset.like_count.mean()
avg_reply_fav_retweet_count = stat_dataset.retweet_count_y.mean()

In [24]:
x_id = set(dataset.id)

In [25]:
# for i,r in org_tweet.iterrows():
#     if r.id not in x_id:
#         print(r.url)
has_replies = []
reply_count = []
reply_fav = []
reply_retweet = []
for i,v in org_tweet.iterrows():
    valid = v.id in x_id
    has_replies.append(valid)
    if valid:
        reply_count.append(number_of_replies[v.id])
        reply_fav.append(avg_reply_fav_count[v.id])
        reply_retweet.append(avg_reply_fav_retweet_count[v.id])
    
org_tweet = org_tweet[has_replies]
# print(org_tweet.shape,len(reply_count))
org_tweet['reply_count'] = reply_count
org_tweet['avg_reply_retweet'] = reply_retweet
org_tweet['avg_reply_fav'] = reply_fav

In [26]:
org_tweet.head()

,id,text,screen_name,retweet_count,favorite_count,friends_count,followers_count,url,RT,original_tweeter,emojis_used,reply_count,avg_reply_retweet,avg_reply_fav
0,937247505208827904,Eden Hazard says the England and Belgium playe...,SPonTheBall,35,0,81,25,https://t.co/0Ekz6haggd,True,BBCMOTD,NaN,6,0.000000,0.500000
1,937247428973088768,Legend! #SCFC,boothen_talers,396,0,294,287,https://t.co/NF4w2eDNXp,True,stokecity,👔😎🔴⚪️,17,0.058824,2.176471
2,937246905980952576,And South Korea round off Group F! What do yo...,naneth_mandagi,1181,0,181,65,https://t.co/nCqErE1w9P,True,DFB_Team_EN,🇰🇷,17,0.529412,14.647059
3,937246600803618816,When @matshummels and @os_alanis play against ...,Drizzy__Fake,1582,0,109,306,https://t.co/udLBbxZPKM,True,FCBayernUS,NaN,18,0.833333,7.944444
4,937245866456428545,When you find out you gotta play @DeBruyneKev ...,egn_ho,3313,0,518,643,https://t.co/d1LQSMB5Nu,True,kylewalker2,NaN,17,0.823529,82.176471


In [27]:
org_tweet.shape

(77382, 14)

In [28]:
org_tweet.to_csv('WorldCupDraw_Cleansed_final.csv',encoding='utf8',index=False)

In [35]:
org_tweet.isnull().sum()

id                       0
text                     0
screen_name              0
retweet_count            0
favorite_count           0
friends_count            0
followers_count          0
url                      0
RT                       0
original_tweeter      6833
emojis_used          43533
reply_count              0
avg_reply_retweet        0
avg_reply_fav            0
dtype: int64

In [36]:
df.isnull().sum()

id                       0
text                     0
emojis_used         850526
retweet_count            0
like_count               0
RT                       0
original_tweeter         0
dtype: int64

In [38]:
org_tweet.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77382 entries, 0 to 106547
Data columns (total 14 columns):
id                   77382 non-null int64
text                 77382 non-null object
screen_name          77382 non-null object
retweet_count        77382 non-null int64
favorite_count       77382 non-null int64
friends_count        77382 non-null int64
followers_count      77382 non-null int64
url                  77382 non-null object
RT                   77382 non-null bool
original_tweeter     70549 non-null object
emojis_used          33849 non-null object
reply_count          77382 non-null int64
avg_reply_retweet    77382 non-null float64
avg_reply_fav        77382 non-null float64
dtypes: bool(1), float64(2), int64(6), object(5)
memory usage: 33.5 MB


In [1]:
org_tweet.head()

NameError: name 'org_tweet' is not defined